# 本セクションの目次
1. Avroフォーマット
2. 前方互換と後方互換と完全互換
3. メッセージキューとAvroを連携してみよう
4. Avroファイルの読み書き
5. Avroで前方互換をやってみよう

# カラムナーフォーマット
ここまでで読み込んだJsonやCsvはローデータとよばれる、データ基盤では扱いづらい部類に入るファイルです。  
扱いづらい理由はファイルがスプリッタブルかどうか？という点が関わっています。  

スプリッタブルであればデータを複数端末で分割して処理することが可能ですし、そうでなければ分割して処理をすることができないため非効率が発生します。  

カラムナーフォーマットと次のレクチャーで紹介するAvro(行指向フォーマット)はいずれもスプリッタブルなファイルです。  
そのため、CSVやJSON形式のままデータ基盤で活用を続けるのは悪手なので、必ずカラムナーフォーマット、行指向フォーマットへ変換する処理を挟みましょう。  

カラムナーフォーマットとは、分析に特化したファイル形式です。  
列ごとにデータをまとめて保存することによって、データを圧縮し検索速度(特にGroup byなどの集計構文)をあげています。  
また、カラムナーフォーマットは内部にカラム情報を保持しているためセクション2で紹介したスキーマオンリード機能を使ってスキーマ定義を読み出すことができます。

# 圧縮形式とファイルフォーマット

前のレクチャーではスプリッタブルかどうかが重要であるとお伝えしました。  
ここで一度、ファイルと圧縮形式の組み合わせについてみていきましょう。

```
        paruqet avro csv/json
gz       Y      Y      N
snappy   Y      Y      -
bz2      -      -      Y
圧縮なし  Y      Y      N

```

大事なのは、csvやjsonの場合はbz2を利用しましょう。

# 行指向フォーマット
先ほどはカラムナーフォーマットを紹介しましたが、次は行指向フォーマットを紹介について紹介していきます。  

よく利用されているRDBも行指向の仕組みを持ったデータシステムです。  
RDBはレコードの追加が得意です。  
 
そして、ビッグデータはレコードを一件つづ追加していく様な処理が苦手でしたが行指向フォーマットのAvroの登場によってレコードの追加が頻繁に発生する処理  
つまりストリーミング処理にも適用することが可能になりました。  

まず覚えてほしいのは、Avroはメインとしてストリーミング処理の利用に向いているということです（バッチ処理にも使うことが可能です）  


ビッグデータにおけるストリーミングについて詳しく知りたい方は  
「データサイエンスのためのストリーミング前処理入門　PythonとSparkで始めるビッグデータストリーミング処理入門」
を是非受講ください。

In [ ]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init("/home/pyspark/spark")

In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .config("spark.jars.packages", "org.apache.spark:spark-streaming_2.13:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-avro_2.12:3.2.0") \
    .enableHiveSupport() \
    .getOrCreate()

# パッケージを複数渡したい時は「,」で繋いで渡します。
# Sparkのバージョンにしっかりと合わせます(今回はSparkのバージョンが3.2を使っています。)。

# Dataframeとは？

データを枠組み(フレーム)として扱うことができるのがDataFrameです。  
データフレームの作成方法は

- SQLから生成する
- データを読み込んで生成する
- プログラム的に作成する

SQLから生成はSQLで取得した結果がデータフレームで返却されるます  

データを読み込んで生成する場合はCSVやJSon、Parquet、Avroなどのファイルを読み込んでデータフレームに取り込みを行うことが可能  

プログラム的に作成するのは、空のデータフレームが欲しい時など

# ビッグデータの世界のDDL

ビッグデータの世界でのDDLはRDSと同じ様にDDL文を実行することが可能です。  
今回は以下のDDLについてみていきましょう  

- Create Database 文
- CREATE EXTERNAL TABLE文
- CREATE VIEW
- ADD PARTITION(MSCK REPAIR)

## CREATE DATABASE文
データベースの作成を行います。
こちらはRDSなどのCreate Database　と同じ方法で作成が可能です。

In [ ]:
spark.sql("create database if not exists super_crush_course")

## CREATE EXTERNAL TABLE文
テーブル定義の構成要素をみていきましょう

```
CREATE EXTERNAL TABLE IF NOT EXISTS super_crush_course.csv_table ( id INT, date STRING)
PARTITIONED BY (dt INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LOCATION '/home/pyspark/super_crush_course.db/csv_table/dataset/parquet/';

#S3などであれば、以下のように設定を変えることも可能です。
LOCATION 's3://data.platform/super_crush_course.db/raw_zone/sampletable/';

```

ビッグデータの世界では、実データとデータベース定義/テーブル定義(メタデータ)は明確に分離されています。  
今回のコースだと実データはローカル端末のSSD(HDD)でテーブル定義はMysqlに登録されています。  
明確に分離されているからこそ、場所を指し示す宣言であるLocationが必要になってきます  

ロケーションは「super_crush_course.db」とDB名.db/TABLE名とすることが通例です。  
Externalは外部のという意味で、オンプレ環境の場合はつけないことが多かったが、クラウド環境ではつけるのが必須となっている設定。

メタデータについてさらに詳しく知りたい方は、以下の記事を参照してみてください  
「【PythonとSparkで始めるデータマネジメント入門】 ビッグデータレイクのための統合メタデータ管理入門」

上記の例は、CSV形式のテーブルです。  
それ以外にもParquet形式、Avro形式でテーブルを作成することができます。

```
# メタデータ保存用のSparkテーブル
CREATE TABLE IF NOT EXISTS super_crush_course.parquet_table 
(code String, gengo String,wareki String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)
PARTITIONED BY (kenmei String)
STORED AS PARQUET
TBLPROPERTIES ("parquet.compression"="SNAPPY");
LOCATION '/home/pyspark/super_crush_course.db/parquet_table';

```

圧縮形式と保存するファイルフォーマットを指定してテーブルの作成を行なっていきます。  
ちなみにテーブル定義のカラムは、今回読み込んだCSV/JSONのスキーマになります。  
Partitionとはデータの区切りで　 

In [ ]:
## テーブル作成
# テーブル作成も同様に、spark.sqlを使ってテーブルを作成していきます。

spark.sql("""
CREATE TABLE IF NOT EXISTS super_crush_course.parquet_table 
(code String, gengo String,wareki String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)
PARTITIONED BY (kenmei String)
STORED AS PARQUET
TBLPROPERTIES ("parquet.compression"="SNAPPY");
LOCATION '/home/pyspark/super_crush_course.db/parquet_table';
""")

# Create View
ビューを作成します。
ビューとは、仮想的なテーブルのことでデータを生成しなくてもテーブルを生成することが可能です。

言葉で伝えるより実際に見た方がいいと思うので、早速作ってみましょう。

In [ ]:
# create viewを実演を行う

# テンポラリテーブル
テンポラリーテーブルとは一時的にテーブルを作成することで、SparkSessionごとに生成が可能です。

特にスキーマオンリードで読み込んだdataframeを一時的にテーブルにすることで、SQLでの操作を可能にすることができます。

一連の流れを見てみましょう

In [ ]:
# テンポラリーテーブルの実演

# ビッグデータ世界のDMLとは

ビッグデータの世界のSQLは基本的にSQL’ライク’です。  
というのもRDSのSQLを前提にしてライクと言っているだけなので、ビッグデータ世界を中心としたらSQLそのものです。  
RDSでのSQLに慣れている人は、ビッグデータの世界のSQLは難なくこなすことができると思います。  

- SELECT
- INSERT?
- UPDATE?
- DELETE?
- CTAS
- SELECT INSERT

今回は4つのDMLを確認してみます。

In [ ]:
spark.sql("""show tables in super_crush_course""")

# 分析関数の練習をしよう

# ピボットテーブル

# LAG関数

In [ ]:
spark.stop()